![banner_e](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_automatizacion.gif?raw=true)

**<mark style="background:#2bfe9c">Script de automatización</mark> proviene del Notebook <mark>Pre ETL Yelp Business</mark>**

In [1]:
pip install geopandas

StatementMeta(, 55949e97-890f-4b87-9e63-67e4a117ec82, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 85.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 112.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import geopandas as gpd                     
from shapely.geometry import Point
from geopy.geocoders import Nominatim

StatementMeta(, 55949e97-890f-4b87-9e63-67e4a117ec82, 4, Finished, Available)

Importar en un dataframe el archivo original _business.pkl_ desde la ruta de acceso en el ordenador donde se encuentran alojado.

In [3]:
df_business = pickle.load(open("/lakehouse/default/Files/data/original/Yelp/business.pkl", 'rb')) #leer archivo .pkl
pd.options.display.max_columns=0 #mostrar todas las columnas
df_business = df_business.iloc[:,0:14] #quedarse con las columnas necesarias

StatementMeta(, 55949e97-890f-4b87-9e63-67e4a117ec82, 5, Finished, Available)

Se comienza eliminando los registros _NaN_ presentes en la columna _categories_, luego mediante el uso de una función se extraen los registros de todo el dataframe que cumplan con la condición de que en la columna _categories_ aparezca la palabra _Hotels_ en la primera o segunda posición; esto asegura que el rubro principal del comercio sea este o semejante (casa de alojamiento, hostal, etc).

In [4]:
df_business = df_business.dropna(subset=['categories']) #eliminar los registros NaN de la columna categories

def filter_categories_hotels(categories:list): #definir la funcion (recibe una lista como parámetro)
    for item in categories[0:2]: #validar que la palabra 'Hotels' aparezca en la 1era o 2da posición de la lista
        if type(item) == str and item.strip() == 'Hotels':
            return True   
    return False

df_business['categories'] = df_business['categories'].str.split(',') #crear una lista a partir de los valores de la columna 'categories' 
df_business = df_business[df_business['categories'].apply(filter_categories_hotels)] #aplicar la función en la columna 'categories'

StatementMeta(, 55949e97-890f-4b87-9e63-67e4a117ec82, 6, Finished, Available)

Luego de tener tener filtrado el dataset para el rubro de hoteles se procede a eliminar los registros que no se encuentren en EEUU. Esto se realiza usando la librería _GeoPy_ y su geocodificador _Nominatim_, el cual trae la dirección completa de los hoteles a través de la latitud y de la longitud suministradas (que se encuentran en el dataset).

In [5]:
def search_state(lati_long:str): #definir la función que retornará la ubicación de cada hotel (recibe latitud y longitud como parámetros)
    geolocalizador = Nominatim(user_agent="aaaa")
    ubicacion = geolocalizador.reverse(lati_long)
    return ubicacion.address.split(',') #devuelve una lista con la dirección completa del hotel

df_business = df_business.astype({'latitude':'str', 'longitude':'str'}) #convertir las columnas 'latitude' y 'longitude' a tipo string
df_business['lati_long'] = df_business['latitude'] + ',' + df_business['longitude'] #unir en una sola columna los valores de 'latitude' y 'longitude'
df_business['state_en'] = df_business['lati_long'].apply(search_state) #guardar en la columna 'state_en' lo que retorna la función

StatementMeta(, 55949e97-890f-4b87-9e63-67e4a117ec82, 7, Finished, Available)

Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /reverse?lat=35.9506307027&lon=-86.8082302809&format=json&addressdetails=1
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /reverse?lat=39.6540342864&lon=-86.4005740732&format=json&addressdetails=1
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /reverse?lat=32.2190283&lon=-110.9699202&format=json&addressdetails=1
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connecti

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=39.9123688&lon=-86.2241473&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

Se realiza una copia del dataframe anterior dado que el servicio de _GeoPy_ es limitado. Así se puede sobreescribir el nuevo dataframe sin afectar el original.

In [ ]:
df_b = df_business.copy()

StatementMeta(, , , Cancelled, )

Por otra parte se importa en un dataframe los 50 estados de los EUUU escritos en español e inglés así como su abreviatura.

In [ ]:
df_state_EEUU = pd.read_csv("/lakehouse/default/Files/data/original/Yelp/estados_EEUU.csv")
df_state_EEUU = df_state_EEUU.iloc[:,0:3]

StatementMeta(, , , Cancelled, )

Dado que se tiene la dirección completa del hotel y solo se desea tener el estado en el cual se encuentra el hotel se crea una función que devuelva el estado luego de cruzar la información contenida en las columnas _state-en_ de los dataframes _df-state-EEUU_ y _df-b_.

In [ ]:
lista_state_EEUU_en = df_state_EEUU['state_en'].to_list() #convertir la columna 'state_en' en una lista

def search_state(fila_state_en:list): #definir la función
    fila_state_en = fila_state_en

    for item in fila_state_en:
        if item.strip() in lista_state_EEUU_en: #buscar cada item de la dirección en la lista de estados
            return item.strip() #devuelve el estado sin espacios en blanco

df_b['state_en'] = df_b['state_en'].apply(lambda x: search_state(x)) #guardar en la columna 'state_en' lo que devuelve la función

StatementMeta(, , , Cancelled, )

A continuación se puede observar la cantidad de valores nulos que existen aún en el dataframe.

In [ ]:
df_b.info()

StatementMeta(, , , Cancelled, )

Los valores nulos que existen en la columna _state-en_ corresponden a registros de hoteles que no son de EEUU. Por lo tanto se procede a elminarlos y además esta columna se cruza con la del dataframe de los estados para traer estos en idioma español y tener un mejor entendimiento.

In [ ]:
df_b = df_b.dropna(subset=["state_en"]) #eliminar valores nulos (registros fuera de EEUU)
df_b = pd.merge(df_b, df_state_EEUU.iloc[:,0:2], on='state_en', how='left') #traer estados en ingles

StatementMeta(, , , Cancelled, )

Como último paso antes de exportar el dataframe se procede a eliminar las columnas que no se necesitarán y a renombrar la columna 'state-sp'.

In [ ]:
df_b = df_b.drop(['state','lati_long','state_en'], axis=1) #eliminar columnas
df_b = df_b.rename(columns={'state_sp':'state'}) #renombrar columna

StatementMeta(, , , Cancelled, )

Exportar el dataframe con la data filtrada para el rubro _hotel_ y peteneciente solo a EEUU.

In [ ]:
df_b.to_csv("/lakehouse/default/Files/data/beta/Yelp/business.csv")

StatementMeta(, , , Cancelled, )